# Chapter 5: Monte Carlo Methods

## 1. Introduction
- Do not assume complete knownledge of the environment
- Learning from experience of interaction with environment
    - Experience is divided into **episodes**
    - Based on averaging sample returns - complete returns of each episode
- Like an associative bandit
    - Nonstationary from the point of view of the earlier state
- Adapt the idea of GPI from DP

## 2. Monte Carlo Prediction
- Learning the state-value function $v_\pi(s)$ for a given policy $\pi$
- Estimate from experience by averaging the returns observed after visits to that state $s$
- a **visit** to $s$: each occurrence of state $s$ in an episode
- 2 methods:
    - **first-visit** MC: average of the returns only for the first visits to $s$
    - **every-visit** MC: average of the returns for all visits to $s$

- Only on choice considered at each state (unlike DP) - only sampled on the one episode
- Estimates for each state are independent
    - computational expense of estimating a single state is independent of the number of states
- Do not bootstrap (unlike DP)

- first-visit MC algorithm

![First-Visit MC](assets/5.1.first-visit-mc.png)

## 3. Monte Carlo Estimation of Action Values
- if a model is not available, MC is useful to estimate action values $q_*$
- Averaging returns starting from state $s$, taking action $a$ following  policy $\pi$
- a **visit** to pair $s,a$: each occurrence of state $s$ and action $a$ is taken in it,  in an episode
- 2 methods:
    - first-visit MC: average of the returns only for the first visits to $s, a$
    - every-visit MC: average of the returns for all visits to $s, a$
- Need to estimate the value of all the actions from each state
- **Exploring starts**: every pairs $s,a$ has nonzero probability of being selected as the start

## 4. Monte Carlo Control
- Use GPI
$$\pi_0 \stackrel{E}{\longrightarrow} q_{\pi_0} \stackrel{I}{\longrightarrow} \pi_1 \stackrel{E}{\longrightarrow} q_{\pi_1} \stackrel{I}{\longrightarrow} \pi_2 \stackrel{E}{\longrightarrow} ... \stackrel{I}{\longrightarrow} \pi_* \stackrel{E}{\longrightarrow} q_{\pi_*}$$

- Policy evaluation $\stackrel{E}{\longrightarrow}$: using MC methods for prediction
- Policy improment $\stackrel{I}{\longrightarrow}$: policy greedy with respect to the current value function
    - meets the conditions for policy improvement by policy improvement theorem
    - if, $\pi_{k+1}=\pi_k$, then $\pi_k=\pi_*$
$$
\begin{aligned}
q_{\pi_k}\big(s,\pi_{k+1}(s)\big) &= q_{\pi_k}\big(s,\arg\max_a q_{\pi_k}(s,a)\big)
\\ &= \max_a q_{\pi_k}(s,a)
\\ &\ge q_{\pi_k}\big(s,\pi_k(s)\big)
\\ &\ge v_{\pi_k}(s)
\end{aligned}
$$

- Converage conditions assumptions:
    - (1) episodes have exploring starts
    - (2) policy evaluation could be done with an infinite number of episodes
 
- Need to remove both assumptions in order to obtain a practical algorithm
- Solve the assumption (2):
    - Obtain bounds on the magnitude and probability of error in the estimates, assure that these bounds are sufficiently small
    - Value Iteration
    - Alterate between improvement and evaluation steps for single states
    
- Monte Carlo Exploring Starts (*Monte Carlo ES*)
    - alterate between evaluation and improvement on an episode-by-episode basis
    - convergence to this fixed point (fixed point is optimal policy $\pi_*$) seems inevitable

![Monte Carlo Exploring Starts](assets/5.3.mc-es.png)

- Monte Carlo without Exploring Starts
    - **on-policy** methods: evaluate or improve the policy that is used to make decisions
    - **off-policy** methods: evaluate or improve the policy different from that is used to generate the data

## 5. On-Policy method
- Learn about policy currently executing
- Policy is generally soft $\pi(a | s) > 0$
- ε-soft policy like ε-greedy:
    - probability of nongreedy is $\dfrac{\epsilon}{| \mathcal A(s) |}$
    - and, probability of greedy is $1-\epsilon+\dfrac{\epsilon}{| \mathcal A(s) |}$

- ε-greedy with respect to $q_\pi$ is an improvement over any ε-soft policy $\pi$ is assured by the policy improvement theorem
$$
\begin{aligned}
q_\pi\big(s,\pi'(s)\big) &= \sum_a \pi'(a | s) q_\pi(s,a)
\\ &= \frac{\epsilon}{| \mathcal A(s) |}\sum_a q_\pi(s,a) + (1-\epsilon)\max_a q_\pi(s,a)
\\ &\ge \frac{\epsilon}{| \mathcal A(s) |}\sum_a q_\pi(s,a) + (1-\epsilon)\sum_a \frac{\pi(a | s)-\frac{\epsilon}{| \mathcal A(s) |}}{1-\epsilon} q_\pi(s,a)
\\ &= \frac{\epsilon}{| \mathcal A(s) |}\sum_a q_\pi(s,a) - \frac{\epsilon}{| \mathcal A(s) |}\sum_a q_\pi(s,a) + \sum_a \pi(a | s) q_\pi(s,a)
\\ &\ge v_\pi(s)
\end{aligned}
$$

- Converages to best ε-soft policy $v_\pi=\tilde v_* ~~~, \forall s\in\mathcal S$

![On-Policy e-soft](assets/5.4.e-soft.png)

## 6. Off-policy method
- learn the value of the **target policy** $\pi$ from experience due to **behavior policy** $b$
    - optimal policy: target policy
    - exploratory & generate policy: behavior policy
- more powerfull and general than on-policy
- greater variance and slower to converge
- on-policy methods is special case in which $\pi = b$
- Coverage assumption: $b$ generates behavior that covers, or includes, $\pi$
    $$\pi(a | s) > 0 \implies b(a | s) > 0$$
- Method: use **importance sampling**
    - estimate expected values under one distribution given samples from another
    - **importance-sampling ratio**: weighting returns according to the relative probability of their trajectories under the two policies  
- The relative probability of the trajectory under 2 polices depend only on the 2 policies and the sequence:
    $$\rho_{t:T-1} = \prod_{k=1}^{T-1}\frac{\pi(A_k | S_k)}{b(A_k | S_k)}$$
- Expected value of target policy:
    $$v_\pi(s) = E\big[\rho_{t:T-1}G_t | S_t=s\big]$$
    where, $G_t$ is returns of $b$ : $v_b(s) = E\big[G_t | S_t=s\big]$
- indexing time steps in a way that increases across episode boundaries
    - first episode ends in terminal state at time $t-1=100$
    - next episode begins at time $t=101$
- 2 types of importance sampling:
    - **ordinary importance sampling**:
        $$V(s) = \frac{\sum_{t\in\mathscr T(s)}\rho_{t:T(t)-1}G_t}{| \mathscr T |}$$
    - **weighted importance sampling**:
        $$V(s) = \frac{\sum_{t\in\mathscr T(s)}\rho_{t:T(t)-1}G_t}{\sum_{t\in\mathscr T(s)}\rho_{t:T(t)-1}}$$
        
    where:
        - $\mathscr T(s)$: set of all time steps in which state $s$ is visited
        - $T(t)$: first time of termination following time $t$
        - $G_t$: returns after $t$ up through $T(t)$
        - $\{G_t\}_{t\in\mathscr T(s)}$ are the returns that pertain to state $s$
        - $\{\rho_{t:T(t)-1}\}_{t\in\mathscr T(s)}$ are the corresponding importance-sampling ratios
        
- for the *first-visit* methods:
    - ordinary importance sampling is unbiased and variance is unbounded
    - weighted importance sampling is biased and variance is bound $[0, 1)$ (converges to zero)
- for the *every-visit* methods:
    - both biased
    - bias falls asymptotically to zero as the number of samples increases

## 7. Incremental Implementation for MC prediction
- similar to Bandits ([Chap 2](chap2.ipynb#5.-Incremental-Implementation)) but for the average *returns* $G_t$ instead of average *rewards* $R_t$
- for *on-policy*: exactly the same methods as Bandits
- for *ordinary importance sampling*
    - scaling returns by $\rho_{t:T(t)-1}$
- for *weighted importance sampling*
    - have sequence of returns $G_1, G_2, ..., G_{n-1}$ all starting in the same state
    - corresponding random weight $W_i$ (e.g., $W_i = \rho_{t_i:T(t_i)-1}$)
        $$V_n = \frac{\sum_{k=1}^{n-1}W_kG_k}{\sum_{k=1}^{n-1}W_k} ~~~, n\ge 2$$
     - update rule:
         $$
         \begin{cases}
         V_{n+1} &= V_n + \dfrac{W_n}{C_n}\big[G_n - V_n\big] ~~~, n\ge 1
         \\
         C_{n+1} &= C_n + W_{n+1} ~~~, C_0 = 0
         \end{cases}
         $$
     - can apply to on-policy when $\pi=b, W=1$
     
     ![Weighted Importance Sampling](assets/5.6.weighted-importance-sampling.png)

## 8. Off-policy MC Control
- Requires behavior $b$ is soft: $b(a | s) > 0$
- Advantage:
    - target policy $\pi$ may be deterministic (e.g., greedy)
    - while the behavior policy $b$ can continue to sample all possible actions
- Disadvantages:
    - learn only from the tails of episodes (the remaining actions in the episode are greedy)
    - Greatly slow learning when non-greedy actions are common (states appearing in the early portions of long episodes)

![off-policy MC control](assets/5.7.off-policy-mc-control.png)

## 9. Discounting-aware Importance Sampling
- Use discounted rewards structure of the returns to reduce the variance of off-policy estimators
- **flat partial returns**:
    $$\overline G_{t:h} = R_{t+1} + R_{t+2} + ... + R_h ~~~, 0\le t < h\le T$$
- full return $G_t$ by flat partial returns:
$$
\begin{aligned}
G_t &= R_{t+1} + \gamma R_{t+2}  + \gamma^2 R_{t+3} + ... + \gamma^{T-t-1} R_T
\\ &= (1-\gamma)R_{t+1}
\\ & ~~~ + (1-\gamma)\gamma(R_{t+1} + R_{t+2})
\\ & ~~~ + (1-\gamma)\gamma^2(R_{t+1} + R_{t+2} + R_{t+3})
\\ & ~~~\vdots
\\ & ~~~ + (1-\gamma)\gamma^{T-t-2}(R_{t+1} + R_{t+2} + ... + R_{T-1})
\\ & ~~~ + \gamma^{T-t-1}(R_{t+1} + R_{t+2} + ... + R_T)
\\ &= (1-\gamma)\sum_{h=t+1}^{T-1}\gamma^{h-t-1}\overline G_{t:h} + \gamma^{T-t-1}\overline G_{t:T}
\end{aligned}
$$

- **discounting-aware** importance sampling
    - discount rate but have no effect if $\gamma=1$
    - for ordinary importance-sampling estimator:
      $$V(s)=\dfrac{\displaystyle\sum_{t\in\mathscr T(s)}\Big( (1-\gamma)\sum_{h=t+1}^{T(t)-1}\gamma^{h-t-1}\rho_{t:h-1}\overline G_{t:h} + \gamma^{T(t)-t-1}\rho_{t:T(t)-1}\overline G_{t:T(t)}\Big)}{| \mathscr T(s) |}$$
    - for weighted importance-sampling estimator:
       $$V(s)=\dfrac{\displaystyle\sum_{t\in\mathscr T(s)}\Big( (1-\gamma)\sum_{h=t+1}^{T(t)-1}\gamma^{h-t-1}\rho_{t:h-1}\overline G_{t:h} + \gamma^{T(t)-t-1}\rho_{t:T(t)-1}\overline G_{t:T(t)}\Big)}{\displaystyle\sum_{t\in\mathscr T(s)}\Big( (1-\gamma)\sum_{h=t+1}^{T(t)-1}\gamma^{h-t-1}\rho_{t:h-1} + \gamma^{T(t)-t-1}\rho_{t:T(t)-1}\Big)}$$

## 10. Per-decision Importance Sampling
- One more way of reducing variance, even if $\gamma=1$
- Use structure of the returns as sum of rewards
$$
\begin{aligned}
\rho_{t:T-1}G_t &= \rho_{t:T-1}(R_{t+1}+\gamma R_{t+2}+...+\gamma^{T-t-1} R_T)
\\ &= \rho_{t:T-1}R_{t+1}+\gamma\rho_{t:T-1}R_{t+2}+...+\gamma^{T-t-1}\rho_{t:T-1}R_T
\end{aligned}
$$

    where, sub-term $\rho_{t:T-1}R_{t+k}$ depend only on the first and the last rewards
$$E[\rho_{t:T-1}R_{t+k}] = E[\rho_{t:t+k-1}R_{t+k}]$$

- **per-decision** importance-sampling
$$E[\rho_{t:T-1}G_t] = E[\tilde G_t]$$

    where, $\tilde G_t=\rho_{t:t}R_{t+1} + \gamma\rho_{t:t+1}R_{t+2} + \gamma^2\rho_{t:t+2}R_{t+3} + ... + \gamma^{T-t-1}\rho_{t:T-1}R_T$


- Use for *ordinary* importance-sampling
    - same unbiased expectation (in the first-visit case) as the ordinary importance-sampling estimator
    - but not consistent (do not converge to the true value with infinite data)
    $$V(s)=\frac{\sum_{t\in\mathscr T(s)}\tilde G_t}{| \mathscr T(s) |}$$

- NOT for weighted importance-sampling